In [ ]:
!pip install hyperopt
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from hyperopt import hp, fmin, tpe
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import Word2Vec

drive.mount('/content/drive')

In [ ]:
def metrics_classification(y_test, predicted):
    print("Precision : {0:6.5f}".format(precision_score(y_test, predicted)))
    print("Recall: {0:6.5f}".format(recall_score(y_test, predicted)))
    print("F1-measure: {0:6.5f}".format(f1_score(y_test, predicted)))
    print("Accuracy: {0:6.5f}".format(accuracy_score(y_test, predicted)))

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Финальный_датафрейм.csv')
df = df.replace({'score':{7:0,8:1, 9:1, 10:1}})

In [ ]:
df.value_counts()

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [ ]:
model_w2v = Word2Vec(sentences=df['prepared_text'], vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
def document_vector(doc):
    # удаляем слова, которые не входят в словарь модели
    doc = [word for word in doc if word in model_w2v.wv.index_to_key]
    if not doc:
        return np.zeros(model_w2v.vector_size)
    return np.mean(model_w2v.wv[doc], axis=0)

In [ ]:
df['doc_vector'] = df['prepared_text'].apply(document_vector)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( np.array(list(df['doc_vector'])), df['score'], test_size=0.25, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
ros = RandomOverSampler(random_state=0)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.fit_transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def objective(params):
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    if f1<=0.99:
        return -f1
    else:
        return 1.00

space = {
        'n_estimators': hp.choice('n_estimators', [3,5,10,15,20,30,35,40,50]),
        'criterion':  hp.choice('criterion', ['gini', 'entropy', 'log_loss']),
        'max_depth':  hp.choice('max_depth', [3,5,10,20,30,40,50,60,70]),
        'min_samples_leaf': hp.choice('min_samples_leaf', [1,2,3,4,5,6,7,8,9,10])
    }

best_params =fmin(
    fn=objective, space=space, algo=tpe.suggest, max_evals=50)

100%|██████████| 50/50 [00:42<00:00,  1.17trial/s, best loss: -0.869198312236287]


In [ ]:
best_params

{'criterion': 1, 'max_depth': 4, 'min_samples_leaf': 1, 'n_estimators': 5}

In [ ]:
clf = RandomForestClassifier(n_estimators=30,criterion='entropy', max_depth=30, min_samples_leaf=2, class_weight={0:1,1:4})
%time clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
metrics_classification(y_test, y_pred)

CPU times: user 1.52 s, sys: 4.79 ms, total: 1.52 s
Wall time: 1.64 s
Precision : 0.83137
Recall: 0.95918
F1-measure: 0.89071
Accuracy: 0.80905


In [ ]:
def objective(params):
    clf = KNeighborsClassifier(**params)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    if f1<=0.99:
        return -f1
    else:
        return 1.00

space = {
        'n_neighbors': hp.choice('n_neighbors', [2,3,4,5,6]),
        'weights': hp.choice('weights', ['uniform', 'distance'])
    }

best_params =fmin(
    fn=objective, space=space, algo=tpe.suggest, max_evals=50)

100%|██████████| 50/50 [00:09<00:00,  5.23trial/s, best loss: -0.815068493150685]


In [ ]:
best_params

{'n_neighbors': 0, 'weights': 1}

In [ ]:
clf = KNeighborsClassifier(n_neighbors = 2, weights = 'distance')
%time clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
metrics_classification(y_test, y_pred)

CPU times: user 2.62 ms, sys: 0 ns, total: 2.62 ms
Wall time: 2.72 ms
Precision : 0.83146
Recall: 0.80544
F1-measure: 0.81824
Accuracy: 0.70971


In [ ]:
import warnings
warnings.filterwarnings('ignore', 'Line Search failed')

def objective(params):
    params['max_iter'] = 1000
    log_reg = LogisticRegression(**params)
    log_reg.fit(X_train, y_train)

    y_pred = log_reg.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    if f1<=0.99:
        return -f1
    else:
        return 1.00

space = {
        'C': hp.loguniform('C', -3, 3),
        'penalty': hp.choice('penalty', ['l2']),
        'solver': hp.choice('solver',['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'])
    }

best_params =fmin(
    fn=objective, space=space, algo=tpe.suggest, max_evals=50)

  2%|▏         | 1/50 [00:00<00:14,  3.28trial/s, best loss: -0.6000000000000001]

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)



  4%|▍         | 2/50 [00:09<04:38,  5.80s/trial, best loss: -0.7277556440903054]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 16%|█▌        | 8/50 [00:20<01:42,  2.45s/trial, best loss: -0.7344782034346102]

...

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)



 96%|█████████▌| 48/50 [01:19<00:03,  1.63s/trial, best loss: -0.7344782034346102]

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)



100%|██████████| 50/50 [01:24<00:00,  1.69s/trial, best loss: -0.7344782034346102]


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)



In [ ]:
best_params

{'C': 6.327446335364779, 'penalty': 0, 'solver': 3}

In [ ]:
clf = LogisticRegression(C=6.327446335364779, random_state=0, solver='sag', max_iter=1000, class_weight={0:2,1:8})
%time clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
metrics_classification(y_test, y_pred)

CPU times: user 3.85 s, sys: 13.5 ms, total: 3.87 s
Wall time: 4.99 s
Precision : 0.81105
Recall: 0.99864
F1-measure: 0.89512
Accuracy: 0.81015


In [ ]:
def objective(params):
    clf = CatBoostClassifier(**params)
    clf.fit(X_train, y_train)
    print(f'PARAMS = {params}')
    y_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    if f1<=0.99:
        return -f1
    else:
        return 1.00

space = {
        'iterations': hp.choice('iterations', [100,200,300,400,500,600,700]),
        'learning_rate': hp.choice('learning_rate', [0.001, 0.05, 0.1]),
        'depth': hp.choice('depth', [4,5,6,7,8])
    }

best_params =fmin(
    fn=objective, space=space, algo=tpe.suggest, max_evals=10)

Выходные данные были обрезаны до нескольких последних строк (5000).
604:	learn: 0.0128407	total: 1m 20s	remaining: 12.6s

605:	learn: 0.0128166	total: 1m 20s	remaining: 12.4s

606:	learn: 0.0128087	total: 1m 20s	remaining: 12.3s

607:	learn: 0.0127701	total: 1m 20s	remaining: 12.2s

608:	learn: 0.0127579	total: 1m 20s	remaining: 12s

609:	learn: 0.0127127	total: 1m 20s	remaining: 11.9s

610:	learn: 0.0126929	total: 1m 20s	remaining: 11.8s

611:	learn: 0.0126628	total: 1m 20s	remaining: 11.6s

612:	learn: 0.0126350	total: 1m 20s	remaining: 11.5s

613:	learn: 0.0126135	total: 1m 21s	remaining: 11.4s

614:	learn: 0.0125668	total: 1m 21s	remaining: 11.2s

615:	learn: 0.0125219	total: 1m 21s	remaining: 11.1s

616:	learn: 0.0124795	total: 1m 21s	remaining: 11s

617:	learn: 0.0124707	total: 1m 21s	remaining: 10.8s

618:	learn: 0.0124707	total: 1m 21s	remaining: 10.7s

619:	learn: 0.0124404	total: 1m 21s	remaining: 10.5s

620:	learn: 0.0124294	total: 1m 21s	remaining: 10.4s

621:	learn: 0.0124

In [ ]:
best_params

{'depth': 4, 'iterations': 6, 'learning_rate': 2}

In [ ]:
clf = CatBoostClassifier(depth = 8, iterations = 700, learning_rate = 0.1, class_weights={0:2,1:8})
%time clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
metrics_classification(y_test, y_pred)

0:	learn: 0.6506740	total: 352ms	remaining: 4m 6s
1:	learn: 0.6169443	total: 599ms	remaining: 3m 28s
2:	learn: 0.5881136	total: 817ms	remaining: 3m 9s
3:	learn: 0.5654990	total: 1.06s	remaining: 3m 5s
4:	learn: 0.5457173	total: 1.3s	remaining: 3m 1s
5:	learn: 0.5307771	total: 1.45s	remaining: 2m 47s
6:	learn: 0.5168808	total: 1.59s	remaining: 2m 37s
7:	learn: 0.5080404	total: 1.75s	remaining: 2m 31s
8:	learn: 0.4994585	total: 1.92s	remaining: 2m 27s
9:	learn: 0.4933262	total: 2.06s	remaining: 2m 22s
10:	learn: 0.4851981	total: 2.21s	remaining: 2m 18s
11:	learn: 0.4756709	total: 2.35s	remaining: 2m 14s
12:	learn: 0.4728635	total: 2.49s	remaining: 2m 11s
13:	learn: 0.4689644	total: 2.63s	remaining: 2m 9s
14:	learn: 0.4636379	total: 2.75s	remaining: 2m 5s
15:	learn: 0.4571301	total: 2.89s	remaining: 2m 3s
16:	learn: 0.4514922	total: 3.04s	remaining: 2m 2s
17:	learn: 0.4479551	total: 3.17s	remaining: 2m
18:	learn: 0.4454029	total: 3.33s	remaining: 1m 59s
19:	learn: 0.4423963	total: 3.48s	r

In [ ]:
from sklearn.ensemble import VotingClassifier

logreg = LogisticRegression(C=6.327446335364779, random_state=0, solver='sag', max_iter=1000, class_weight={0:2,1:8})
rf = RandomForestClassifier(n_estimators=30,criterion='entropy', max_depth=30, min_samples_leaf=2, class_weight={0:1,1:4})
#knn = KNeighborsClassifier(n_neighbors=2, weights='distance')
catboost = CatBoostClassifier(depth = 8, iterations = 700, learning_rate = 0.1, class_weights={0:2,1:8})

# Создание списка кортежей с моделями
classifiers = [
    ('logistic_regression', logreg),
    ('random_forest', rf),
    #('knn', knn),
    ('catboost', catboost)
]

# Инициализация VotingClassifier
voting_clf = VotingClassifier(estimators=classifiers, voting='soft')  # Используется soft voting для усреднения вероятностей

# Обучение VotingClassifier
voting_clf.fit(X_train, y_train)  # Убедитесь, что у вас есть данные разделенные на тренировочный и тестовый наборы

# Предсказание и оценка модели
y_pred = voting_clf.predict(X_test)
metrics_classification(y_test, y_pred)

0:	learn: 0.6506740	total: 140ms	remaining: 1m 38s
1:	learn: 0.6169443	total: 222ms	remaining: 1m 17s
2:	learn: 0.5881136	total: 301ms	remaining: 1m 9s
3:	learn: 0.5654990	total: 385ms	remaining: 1m 7s
4:	learn: 0.5457173	total: 467ms	remaining: 1m 4s
5:	learn: 0.5307771	total: 546ms	remaining: 1m 3s
6:	learn: 0.5168808	total: 650ms	remaining: 1m 4s
7:	learn: 0.5080404	total: 730ms	remaining: 1m 3s
8:	learn: 0.4994585	total: 819ms	remaining: 1m 2s
9:	learn: 0.4933262	total: 905ms	remaining: 1m 2s
10:	learn: 0.4851981	total: 986ms	remaining: 1m 1s
11:	learn: 0.4756709	total: 1.06s	remaining: 1m 1s
12:	learn: 0.4728635	total: 1.15s	remaining: 1m
13:	learn: 0.4689644	total: 1.24s	remaining: 1m
14:	learn: 0.4636379	total: 1.32s	remaining: 1m
15:	learn: 0.4571301	total: 1.41s	remaining: 1m
16:	learn: 0.4514922	total: 1.49s	remaining: 59.8s
17:	learn: 0.4479551	total: 1.57s	remaining: 59.5s
18:	learn: 0.4454029	total: 1.67s	remaining: 1m
19:	learn: 0.4423963	total: 1.75s	remaining: 59.6s
20: